# Management of Modeling Data

In [1]:
%matplotlib inline

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tellurium as te
import sqlite3

## DataFrame Basics

## Using SQL with DataFrames

In [2]:
conn = sqlite3.connect("example.db")

In [3]:
df = pd.DataFrame({"a":range(10), "b": range(10)})

In [4]:
df.to_sql("table1", conn)


In [5]:
conn.commit()

In [6]:
conn.close()